# Betweenness Centrality
_Betweenness Centrality_ is a way of detecting the amount of influence a node has over the flow of information in a graph.

image::../images/betweenness_centrality.png[]

It is often used to find nodes that serve as a bridge from one part of a graph to another.
In the above example Alice is the main connection in the graph.
If Alice is removed all connections in the graph would be cut off.
This makes Alice "important" because she ensures that no nodes are isolated.

First we'll import the Neo4j driver and Pandas libraries:


In [1]:
from neo4j.v1 import GraphDatabase, basic_auth
import pandas as pd
import os

Next let's create an instance of the Neo4j driver which we'll use to execute our queries.


In [2]:
host = os.environ.get("NEO4J_HOST", "bolt://localhost") 
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver(host, auth=basic_auth(user, password))

Now let's create a sample graph that we'll run the algorithm against.


In [3]:
create_graph_query = '''
CREATE (nAlice:User {id:'Alice'})
,(nBridget:User {id:'Bridget'})
,(nCharles:User {id:'Charles'})
,(nDoug:User {id:'Doug'})
,(nMark:User {id:'Mark'})
,(nMichael:User {id:'Michael'})
CREATE (nAlice)-[:MANAGE]->(nBridget)
,(nAlice)-[:MANAGE]->(nCharles)
,(nAlice)-[:MANAGE]->(nDoug)
,(nMark)-[:MANAGE]->(nAlice)
,(nCharles)-[:MANAGE]->(nMichael);
'''

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_graph_query))
    print("Stats: " + str(result.consume().metadata.get("stats", {})))

Stats: {'labels-added': 6, 'relationships-created': 5, 'nodes-created': 6, 'properties-set': 6}


Finally we can run the algorithm by executing the following query:


In [4]:
streaming_query = """
CALL algo.betweenness.stream('User','MANAGE',{direction:'out'}) 
YIELD nodeId, centrality

MATCH (user:User) WHERE id(user) = nodeId

RETURN user.id AS user,centrality
ORDER BY centrality DESC
LIMIT 20;
"""

with driver.session() as session:
    result = session.read_transaction(lambda tx: tx.run(streaming_query))        
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())

df

,user,centrality
0,Alice,4.0
1,Charles,2.0
2,Bridget,0.0
3,Doug,0.0
4,Mark,0.0
5,Michael,0.0


We can see that Alice is the main broker in this network and Charles is a minor broker.
The others don't have any influence because all the shortest paths between pairs of people go via Alice or Charles.